In [7]:
import pickle as pkl
import pandas as pd

In [2]:
infile = open('real_estate_data_frame','rb')
frame = pkl.load(infile)
infile.close()

In [3]:
frame

,localization_y,localization_x,area,sewers,road,fence,water,electricity,gas,price
0,51.835925,19.193010,1004.0,brak,utwardzana,brak,wodociąg w drodze,w drodze,brak,90360.0
1,50.472433,19.233789,18.0,szambo,asfaltowa,żywopłot,wodociąg na działce,na działce,brak,229000.0
2,51.801330,19.189772,1200.0,brak,utwardzana,brak,brak,brak,brak,84000.0
3,53.723500,20.867400,1262.0,brak,utwardzana,brak,własne ujęcie/studnia,brak,brak,88340.0
4,52.094000,20.807400,950.0,brak,utwardzana,brak,brak,brak,brak,295000.0
...,...,...,...,...,...,...,...,...,...,...
7624,49.939618,19.889479,2000.0,brak,polna,brak,brak,brak,brak,360000.0
7625,49.907755,20.200424,35.2,brak,polna,brak,brak,brak,brak,315000.0
7626,50.026720,19.700051,4400.0,szambo,polna,brak,brak,brak,brak,160000.0
7627,49.987773,19.679529,4400.0,brak,polna,brak,brak,brak,brak,440000.0


In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7629 entries, 0 to 7628
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   localization_y  7629 non-null   float64
 1   localization_x  7629 non-null   float64
 2   area            7629 non-null   float64
 3   sewers          7629 non-null   object 
 4   road            7629 non-null   object 
 5   fence           7629 non-null   object 
 6   water           7629 non-null   object 
 7   electricity     7629 non-null   object 
 8   gas             7629 non-null   object 
 9   price           7629 non-null   float64
dtypes: float64(4), object(6)
memory usage: 596.1+ KB


## creating dummy variables

In [8]:
dummies = pd.get_dummies(frame[['sewers','road','fence','water','electricity','gas']],drop_first=True)
frame = frame.drop(['sewers','road','fence','water','electricity','gas'],axis=1)
frame = pd.concat([frame,dummies],axis=1)

In [10]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7629 entries, 0 to 7628
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   localization_y                           7629 non-null   float64
 1   localization_x                           7629 non-null   float64
 2   area                                     7629 non-null   float64
 3   price                                    7629 non-null   float64
 4   sewers_miejska                           7629 non-null   uint8  
 5   sewers_przydomowa oczyszczalnia ścieków  7629 non-null   uint8  
 6   sewers_szambo                            7629 non-null   uint8  
 7   road_kostka brukowa                      7629 non-null   uint8  
 8   road_polna                               7629 non-null   uint8  
 9   road_utwardzana                          7629 non-null   uint8  
 10  fence_brak                               7629 no

## train test split

In [11]:
X = frame.drop('price',axis=1)
y = frame['price']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## scaling

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler = MinMaxScaler()

In [16]:
X_train= scaler.fit_transform(X_train)

In [17]:
X_test = scaler.transform(X_test)

In [18]:
X_train.shape

(5721, 24)

In [19]:
X_test.shape

(1908, 24)

## creating model 